<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Import data and preprocessing</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [51]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob
import math

In [3]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [4]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [97]:
run ../SRC/config.py

In [31]:
print config_parameters

{'permanent_mapset': 'PERMANENT', 'locationepsg': '31370', 'outputfolder': '../../../Results', 'gisdb': '../../GRASSDATA', 'location': 'WALOUS_31370', 'PYTHONLIB': '/usr/lib/python2.7', 'njobs': 6, 'GISBASE': '/usr/lib/grass76'}


In [20]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/grass76/bin:/usr/lib/grass76/script:/usr/lib/grass76/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2118 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/2046,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/2046 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LIBRARY_PATH = :/usr/lib/grass76/lib 	
COMPIZ_CONFIG_PROFILE = mate 	
WINDOWPATH = 8 	
GTK_OVERLAY_SCROLLING = 0 	
PYTHONPATH

** GRASS GIS Python libraries **

In [9]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

** Other functions**

In [10]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function that change .csv delimiter
from ChangeCsvDelimiter import ChangeCsvDelimiter
# Import function for .gzip archive management
from gzip_management import decompress_gzip
# Import function for computation of proportion of categorical raster in zones (segments)
import compute_proportion_categorical
from compute_proportion_categorical import proportion_class
# Import function for getting list of class of a raster
from data_prep import data_prep

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [11]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [12]:
def import_inputdata(timeclass):
    #Declare empty list that will contain the messages to return
    return_message = []
    
    #Start GRASS session on a new mapset
    message = launch_mapset(timeclass)
    [return_message.append(a) for a in message]
    
    #Import the segmentation layer
    case_study = os.path.split(data["obia_folder"])[-1].lower()
    input_file = os.path.join(data["obia_folder"],"SEGMENTS","segs_%s_%s.tif"%(case_study,timeclass))
    try:
        gscript.run_command('r.in.gdal', overwrite=True, input=input_file, output='segments')
        #gscript.run_command('r.external', overwrite=True, input=input_file, output='segments')
        return_message.append("Segmentation layer has been imported.")
    except:
        return_message.append("ERROR: Segmentation layer has not been imported. Please check for problem.")

    #Define computational region and save it as default
    gscript.run_command('g.region', flags='s', raster='segments')  
    
    #Import the OBIA classification
    input_file = os.path.join(data["obia_folder"],"RASTER","classification_%s_%s.tif"%(case_study,timeclass))
    try:
        gscript.run_command('r.in.gdal', flags='r', overwrite=True, input=input_file, output='obia_clas')
        #gscript.run_command('r.external', overwrite=True, input=input_file, output='obia_clas')
        return_message.append("OBIA classification layer has been imported.")
    except:
        return_message.append("ERROR: OBIA classification layer has not been imported. Please check for problem.")

    #Import the Pixel-based classification
    input_file = data['pix_orth']
    try:
        gscript.run_command('r.in.gdal', flags='r', overwrite=True, input=input_file, output='pixel_clas')
        return_message.append("Pixel-based classification layer has been imported.")
    except:
        return_message.append("ERROR: Pixel-based classification layer has not been imported. Please check for problem.")
     
    #Import the tiles only of a specific 'timeclass'
    condition = "timeclass=%s"%timeclass
    try:
        gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='cutline', 
                            where=condition, key='cat') 
        #gscript.run_command('v.external', overwrite=True, input=tile_file, output='timeclass_%s'%timeclass, where=condition) 
        return_message.append("Tiles with condition 'timeclass=%s' has been imported."%timeclass)
    except:
        return_message.append("ERROR: Tiles has not been imported. Please check for problem."%timeclass)

    # Get a list with unique 'cat' of cutlines tile
    cutlines_cats = list(gscript.parse_command('v.db.select', flags='c', map='cutline', columns='cat'))
            
    #Return
    return return_message, cutlines_cats

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [94]:
# Output folder stat
config_parameters['outputfolder_training_sample'] = os.path.join(config_parameters['outputfolder'],"Training_sample")
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_training_sample'])

The folder '../../../Results/Training_sample' has been created


# Create mapsets with input data for the processing

## OBIA land cover

In [13]:
# Create mapset 
launch_mapset("OBIA")

["Location 'WALOUS_31370' already exist",
 "'OBIA' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/OBIA'"]

In [14]:
list_of_obia_raster = []
for zone in ("ARLON","CHARLEROI","MARCHE","HERVE"):
    list_file = glob.glob(os.path.join(data['obia_folder'],zone,"SEGMENTS","segs_*.tif"))
    [list_of_obia_raster.append((os.path.splitext(a)[0].split(os.sep)[-1],a)) for a in list_file]

In [15]:
for a in list_of_obia_raster:
    print a

('segs_arlon_147385', '../../../Data/obia_2016/ARLON/SEGMENTS/segs_arlon_147385.tif')
('segs_arlon_147480', '../../../Data/obia_2016/ARLON/SEGMENTS/segs_arlon_147480.tif')
('segs_arlon_147783', '../../../Data/obia_2016/ARLON/SEGMENTS/segs_arlon_147783.tif')
('segs_charleroi_147376', '../../../Data/obia_2016/CHARLEROI/SEGMENTS/segs_charleroi_147376.tif')
('segs_charleroi_147332', '../../../Data/obia_2016/CHARLEROI/SEGMENTS/segs_charleroi_147332.tif')
('segs_charleroi_147333', '../../../Data/obia_2016/CHARLEROI/SEGMENTS/segs_charleroi_147333.tif')
('segs_charleroi_147375', '../../../Data/obia_2016/CHARLEROI/SEGMENTS/segs_charleroi_147375.tif')
('segs_marche_147109', '../../../Data/obia_2016/MARCHE/SEGMENTS/segs_marche_147109.tif')
('segs_herve_146555', '../../../Data/obia_2016/HERVE/SEGMENTS/segs_herve_146555.tif')
('segs_herve_146894', '../../../Data/obia_2016/HERVE/SEGMENTS/segs_herve_146894.tif')
('segs_herve_147343', '../../../Data/obia_2016/HERVE/SEGMENTS/segs_herve_147343.tif')
('s

In [ ]:
# Import individual segmentation raster (for each zone and timeclass)
for rast in list_of_obia_raster:
    gscript.run_command('r.in.gdal', overwrite=True, input=rast[1] , output=rast[0])

In [17]:
# Create virtual raster
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join([a[0] for a in list_of_obia_raster]), 
                    output='segmentation')

0

## Pixel ortho land cover

In [33]:
# Create mapset 
launch_mapset("PIXEL_ORTHO_LC")

["Location 'WALOUS_31370' already exist",
 "'PIXEL_ORTHO_LC' mapset created in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PIXEL_ORTHO_LC'"]

In [40]:
# Define input data
inputs = data['ortho_lc']
# Define tile number (average)
tile_number = 2000

In [ ]:
### Create VRT
# Import raster 
gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0])
# Get raster information
info = gscript.raster_info(input_data[0])
# Tile raster 
gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                    width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                    height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
# Get a list of tiles name
list_of_tile_raster = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
# Create virtual raster
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(list_of_tile_raster), 
                    output=input_data[0])

## Pixel SENTINEL (land cover and Sent2Agri)

In [21]:
# Create mapset 
launch_mapset("PIXEL_SENTINEL")

["Location 'WALOUS_31370' already exist",
 "'PIXEL_SENTINEL' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PIXEL_SENTINEL'"]

In [26]:
# Define input data
inputs = [data['senti_lc'], data['senti2agri']]

# Define tile number (average)
tile_number = 400

# Create VRT
for input_data in inputs:
    # Import raster 
    gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0])
    # Get raster information
    info = gscript.raster_info(input_data[0])
    # Tile raster 
    gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                        width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                        height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
    # Get a list of tiles name
    list_of_tile_raster = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
    # Create virtual raster
    gscript.run_command('r.buildvrt', overwrite=True, 
                        input=",".join(list_of_tile_raster),
                        output=input_data[0])

## Binaries from vectorial reference data (Squelette vectoriel, SIGEC, masque forestier)

In [98]:
# Create mapset 
launch_mapset("BINARIES")

["Location 'WALOUS_31370' already exist",
 "'BINARIES' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/BINARIES'"]

**SIGEC agricultural plots + GEMBLOUX forest mask**

In [50]:
# Define input data
inputs = [data['binary_agri'], data['binary_forest']]
# Define tile number (average)
tile_number = 600

# Create VRT
for input_data in inputs:
    # Import raster 
    gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0])
    # Get raster information
    info = gscript.raster_info(input_data[0])
    # Tile raster 
    gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                        width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                        height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
    # Get a list of tiles name
    list_of_tile_raster = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
    # Create virtual raster
    gscript.run_command('r.buildvrt', overwrite=True, 
                        input=",".join(list_of_tile_raster),
                        output=input_data[0])

**Vectorial skeletton (road network, hydrological ways, buildings)**

In [99]:
# Define input data
inputs = [data['binary_roadnet'], data['binary_hydro'], data['binary_built']]
# Define tile number (average)
tile_number = 1500

# Create VRT
for input_data in inputs:
    # Import raster 
    gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0])
    # Get raster information
    info = gscript.raster_info(input_data[0])
    # Tile raster 
    gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                        width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                        height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
    # Get a list of tiles name
    list_of_tile_raster = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
    # Create virtual raster
    gscript.run_command('r.buildvrt', overwrite=True, 
                        input=",".join(list_of_tile_raster),
                        output=input_data[0])

## Other data (Tiles vector layer, training points)

In [52]:
# Create mapset 
launch_mapset("PERMANENT")

["Location 'WALOUS_31370' already exist",
 "'PERMANENT' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PERMANENT'"]

In [93]:
# Allow access to other mapsets
gscript.run_command('g.mapsets', mapset='OBIA', operation='add')

0

**Import tiles of OBIA processing**

In [58]:
# Import vector
gscript.run_command('v.in.ogr', overwrite=True, input=data['tiles'][1], output=data['tiles'][0], key='cat')

0

**Import training points, get value of segment ID and export as .csv file**

In [92]:
# Import vector
gscript.run_command('v.in.ogr', overwrite=True, input=data['training_points'][1], output=data['training_points'][0])

0

In [96]:
# Add seg_id in attribute table
gscript.run_command('v.db.addcolumn', map=data['training_points'][0], columns="seg_id integer")
gscript.run_command('g.region', raster='segmentation')
gscript.run_command('v.what.rast', map=data['training_points'][0], raster='segmentation', column='seg_id')
# Export attribute table as .csv
output_file = os.path.join(config_parameters['outputfolder_training_sample'],'training_points')
gscript.run_command('v.db.select', overwrite=True, 
                    map=data['training_points'][0], separator='comma', file=output_file)

0